<a href="https://colab.research.google.com/github/HARASEON/Deep_Learning_tensorflow/blob/main/Day3_tranfer_learning_horses_vs_human.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Computer vision**
* image classiciation
  * CNN, Softmax
* object detection
  * YOLO, R-CNN, Fast R-CNN, Faster R-CNN
* image segmentation
  * VGG U-Net, FCN, SegNet, 구글 DeepLab V3+

**Image dataset**
* Open image dataset v6
* cumstom detector 만들기기
 * git clone https://github.com/EscVM/OIDv4_ToolKit.git
Autoencoder
* 사용목적: 데이터의 편중이 심한 경우 ex. 신용카드 불법사용, 제조업의 불량품검출 등
* 입력층과 출력층을 만들어준다.
* 중간의 hidden layer: encoder, decoder라 부름
* 처음과 나중을 비교하여 anomaly detection에 사용 or 학습데이터 만듦

**XAI**
* 설명가능한 AI
* Deconvolution network :각 단계를 설명하도록 기존 모델을 변형
* 새로운 모델의 개발-원인-결과를 도출하는 과정이 표현가능하도록 설계
* 모델간 비교

**Grad-CAM(Gradient-weighted Class Activation Mapping)**
* CNN에서 분류(classification)했을 때, 정답에 가장 영향을 많이 주었던 부분을 보여줌
* CAM(Class Activation Map): 뷴류한 값이 어떤 뉴런에서 가장 많이 영향을 주었는지를 보여주는 map
-> 마지막 layer가 아니어도 classification activation function을 용도별로 적용가능 (ex. image classification = softmax)


# Horses vs Humans
* 527개 : 사람이미지
* 500개 : 말이미지
* 목표 val_loss < 0.028

# CNN사용

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

dataset_name = 'horses_or_humans'
train_dataset, info = tfds.load(name = dataset_name, split = 'train[:80%]', with_info = True)
valid_dataset, info = tfds.load(name = dataset_name, split = 'train[-20%:]', with_info = True)

# Data preprocessing
def preprocess(features):
  image, label = tf.cast(features['image'], tf.float32) / 255.0, features['label']
  image = tf.image.resize(image, size = (300,300))
  return image, label

batch_size = 32
train_data = train_dataset.map(preprocess).batch(batch_size)
valid_data = valid_dataset.map(preprocess).batch(batch_size)

# Model building
model = Sequential([
    Conv2D(64, (3,3), input_shape = (300, 300, 3), activation = 'relu'),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3), activation = 'relu'),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3), activation = 'relu'),
    MaxPooling2D(2,2),
    Conv2D(16,(3,3), activation = 'relu'),  
    MaxPooling2D(2,2),
    Conv2D(32,(3,3), activation = 'relu'),
    MaxPooling2D(2,2),
    Conv2D(16,(3,3), activation = 'relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation = 'relu'),
    Dense(64, activation = 'relu'),
    Dense(2, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

# model monitoring
early_stopping = EarlyStopping(monitor = 'val_loss', patience=10)
checkpoint_path = 'checkpoint_ckpt'
checkpoint = ModelCheckpoint(checkpoint_path,
                             save_best_only = True,
                             save_weights_only = True,
                             monitor = 'val_loss',
                             verbose =1)

# model learning
history = model.fit(train_data,
                    validation_data = (valid_data),
                    epochs = 50,
                    callbacks = [checkpoint, early_stopping])
model.load_weights(checkpoint_path)
model.save("horses-humans-1.h5")


Epoch 1/50
26/26 [==============================] - ETA: 0s - loss: 0.6946 - accuracy: 0.5328
Epoch 1: val_loss improved from inf to 0.69379, saving model to checkpoint_ckpt
26/26 [==============================] - 15s 83ms/step - loss: 0.6946 - accuracy: 0.5328 - val_loss: 0.6938 - val_accuracy: 0.4341
Epoch 2/50
25/26 [===========================>..] - ETA: 0s - loss: 0.6668 - accuracy: 0.5888
Epoch 2: val_loss improved from 0.69379 to 0.58693, saving model to checkpoint_ckpt
26/26 [==============================] - 1s 27ms/step - loss: 0.6672 - accuracy: 0.5876 - val_loss: 0.5869 - val_accuracy: 0.6878
Epoch 3/50
25/26 [===========================>..] - ETA: 0s - loss: 0.5810 - accuracy: 0.6925
Epoch 3: val_loss improved from 0.58693 to 0.44753, saving model to checkpoint_ckpt
26/26 [==============================] - 1s 27ms/step - loss: 0.5813 - accuracy: 0.6922 - val_loss: 0.4475 - val_accuracy: 0.8098
Epoch 4/50
25/26 [===========================>..] - ETA: 0s - loss: 0.4242 - ac

Transfer model by chat GPT
Prompt: Code horses-vs-human dataset from tensorflow_dataset using transfer learning
Response: This code uses the VGG16 model as a base for transfer learning, and trains a new classifier on top of its pre-trained convolutional layers. The pre-processing steps and data loading are the same as in the original code. The trainable attribute of the VGG16 model is set to False, so that its weights are not updated during training of the new classifier. The new classifier consists of a single Flatten() layer to convert the output of the VGG16 model to a 1D tensor, followed by two Dense() layers with ReLU and sigmoid activations, respectively. The loss function is set to binary_crossentropy since there are only two classes (horses and humans). The rest of the code is mostly the same as in the original code, with the addition of loading the best weights and saving the trained model.

In [ ]:
from tensorflow.keras.applications import VGG16
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# Load the dataset
dataset_name = 'horses_or_humans'
train_dataset, info = tfds.load(name=dataset_name, split='train[:80%]', with_info=True)
valid_dataset, info = tfds.load(name=dataset_name, split='train[-20%:]', with_info=True)

# Define preprocessing function
def preprocess(features):
  image, label = tf.cast(features['image'], tf.float32) / 255.0, features['label']
  image = tf.image.resize(image, size=(300, 300))
  return image, label

# Preprocess the data
batch_size = 32
train_data = train_dataset.map(preprocess).batch(batch_size)
valid_data = valid_dataset.map(preprocess).batch(batch_size)

# Load the VGG16 model and set its layers to non-trainable
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
vgg16_model.trainable = False

# Build the model
model = Sequential([
    vgg16_model,
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Set up model monitoring
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
checkpoint_path = 'checkpoint_ckpt'
checkpoint = ModelCheckpoint(checkpoint_path,
                             save_best_only=True,
                             save_weights_only=True,
                             monitor='val_loss',
                             verbose=1)

# Train the model
history = model.fit(train_data,
                    validation_data=valid_data,
                    epochs=100,
                    callbacks=[checkpoint, early_stopping])

# Load the best weights and save the model
model.load_weights(checkpoint_path)
model.save('horses_vs_human_tr.h5')

Epoch 1/100
25/26 [===========================>..] - ETA: 0s - loss: 0.2275 - accuracy: 0.9200
Epoch 1: val_loss improved from inf to 0.02438, saving model to checkpoint_ckpt
26/26 [==============================] - 3s 65ms/step - loss: 0.2215 - accuracy: 0.9221 - val_loss: 0.0244 - val_accuracy: 0.9902
Epoch 2/100
25/26 [===========================>..] - ETA: 0s - loss: 0.0033 - accuracy: 0.9987
Epoch 2: val_loss improved from 0.02438 to 0.00226, saving model to checkpoint_ckpt
26/26 [==============================] - 1s 55ms/step - loss: 0.0032 - accuracy: 0.9988 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 3/100
25/26 [===========================>..] - ETA: 0s - loss: 7.0735e-04 - accuracy: 1.0000
Epoch 3: val_loss did not improve from 0.00226
26/26 [==============================] - 1s 45ms/step - loss: 6.9221e-04 - accuracy: 1.0000 - val_loss: 0.0081 - val_accuracy: 0.9951
Epoch 4/100
25/26 [===========================>..] - ETA: 0s - loss: 2.3869e-04 - accuracy: 1.0000
Epoch 4